In [1]:
# Include libraries
from ipywidgets import FloatProgress
from IPython.display import display
from warnings import simplefilter
# Stop warnings from printing
simplefilter('ignore')
import matplotlib.pyplot as plt
from copy import deepcopy
from os import listdir
import numpy as np
import cpuinfo
import sys

# Load pytorch
import torch
import torchvision

In [2]:
# Check Available Devices
if torch.cuda.is_available():
    print('Using {0} GPUs:'.format(torch.cuda.device_count()))
    for i in range(torch.cuda.device_count()):
        print('GPU {0}: {1}'.format(i, torch.cuda.get_device_name(i)))
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("Using " + str(cpuinfo.get_cpu_info()['brand']))

Using 2 GPUs:
GPU 0: GeForce GTX 1070
GPU 1: GeForce GTX 1070


In [60]:
# Build a FCN from pretrained Alexnet
class AlexNetFCN(torch.nn.Module):
    # Initialises the FCN
    def __init__(self, numberOfClasses):
        super().__init__()
        
        # Load pretrained AlexNet to copy parameters from
        AlexNetFCN = torchvision.models.alexnet(pretrained = True)
        
        # Downsampling Pathway
        self.downSample1 = torch.nn.Sequential(*list(AlexNetFCN.features.children())[0:5]) # Dim / 8
        self.downSample3 = torch.nn.Sequential(*list(AlexNetFCN.features.children())[6:12]) # Dim / 16
        # Demensionality Reduction to produce a tensor for each class
        self.downSample4 = torch.nn.Sequential(
                torch.nn.Conv2d(256, numberOfClasses, kernel_size = (1,1)), torch.nn.ReLU(True),
                torch.nn.Conv2d(256, numberOfClasses, kernel_size = (1,1)), torch.nn.ReLU(True),
                torch.nn.Conv2d(256, numberOfClasses, kernel_size = (1,1)), torch.nn.ReLU(True)
        ) # Dim / 32
        
        # Upsampling Pathway - Bilinear interpolation
        self.upSample2 = torch.nn.Sequential(torch.nn.Upsample(scale_factor = 2, mode = 'bilinear'))
        self.upSample8 = torch.nn.Sequential(torch.nn.Upsample(scale_factor = 8, mode = 'bilinear'))
                                        
    # Forward path through the network
    def forward(self, inputTensor):
        # Downsampling Pathway
        output1 = self.downSample1(inputTensor)
        output2 = self.downSample3(output1)
        output3 = self.downSample4(output2)
        # Upsampling Pathway
        output16 = self.upSample2(output3) + output2
        output8 = self.upSample2(output16) + output1
        return self.upSample8(output8)

In [61]:
# Instance a FCN base on AlexNet
alexFCN = AlexNetFCN(2)

In [62]:
# Check the network has been modified
print(alexFCN)

AlexNetFCN(
  (downSample1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
  )
  (downSample3): Sequential(
    (0): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace)
  )
  (downSample4): Sequential(
    (0): Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
    (3): ReLU(inplace)
    (4): Conv2d(256, 2, kernel_size=(1, 1), stride=(1, 1))
    (5): ReLU(inplace)
  )
  (upSample2): Sequential(
    (0): Upsample(scale_factor=2, 